# Notebook 3 - Annotation of highly connected metabolomic feature communities.

#### This notebook is a part of a project with the Center for Systems Biology at the University of Iceland. Here, the goal is to run the features from the metabolomic dataset through an automatic annotation procedure using a custom-made function. Then, the putative annotation is used to remove any un-annotated communities from any further downstream analysis. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

## Define functions:

In [183]:
def match_unknown_mz_values_with_adducts(unknown_mz_dict, known_mz_metabolite_dict, adduct_df, ppm_threshold=5):
    matched_results = {}

    for unknown_mz, tupp in unknown_mz_dict.items():
        ion_mode = tupp[1]
        orig_name = tupp[0]
        matched_results[unknown_mz] = []

        for adduct_row in adduct_df.itertuples(index=False):
            adduct, adduct_mass, adduct_ion_mode,multiply_val = adduct_row[0], adduct_row[1], adduct_row[2],adduct_row[3]
            if multiply_val == 1:
                mass_to_check = unknown_mz - adduct_mass
            elif multiply_val == 2:
                mass_to_check = np.divide((unknown_mz - adduct_mass),multiply_val)
            if ion_mode == adduct_ion_mode:
                for known_mz, metabolite_id in known_mz_metabolite_dict.items():
                    ppm_difference = abs(mass_to_check - known_mz) / abs(known_mz) * 1e6
                    if ppm_difference <= ppm_threshold: 
                        matched_results[unknown_mz].append((metabolite_id, orig_name,adduct, ion_mode, ppm_difference))


    return matched_results

def match_unknown_mz_values_with_metabolite_ids(unknown_mz_dict, known_mz_metabolite_dict, adduct_df, id_to_name_dict, ppm_threshold=5):
    
    # Match unknown masses with adducts:
    resss = match_unknown_mz_values_with_adducts(unknown_mz_dict, known_mz_metabolite_dict, adduct_df,ppm_threshold)

    # Create an empty DataFrame
    columns = ['unknown_mz', 'metabolite_id','feature', 'adduct','ionization','mass_diff']
    df_results = pd.DataFrame(columns=columns)

    # Populate the DataFrame
    for unknown_mz, matches in resss.items():
        for match in matches:
            if isinstance(match, list):
                for metabolite_id in match:
                    df_results = pd.concat([df_results, pd.DataFrame({
                        'unknown_mz': [unknown_mz],
                        'metabolite_id': [metabolite_id],
                        'feature':[match[1]],
                        'adduct': [match[2]],
                        'ionization':[match[3]],
                        'mass_diff':[match[4]],


                    })], ignore_index=True)
            else:
                df_results = pd.concat([df_results, pd.DataFrame({
                    'unknown_mz': [unknown_mz],
                    'metabolite_id': [match[0]],
                    'feature':[match[1]],
                    'adduct': [match[2]],
                    'ionization':[match[3]],
                    'mass_diff':[match[4]],
                })], ignore_index=True)

    def get_dict_value(key):
        return id_to_name_dict.get(key, None)            

    # Use explode to expand lists
    df_results = df_results.explode('metabolite_id').reset_index(drop = True)

    # Find matching kegg ids:
    df_results['metabolite_name'] = df_results['metabolite_id'].apply(get_dict_value)
    
    return df_results


def annotate_module_degree(expression_data,module_membership_data,df_modules,all_info_dat,df_mapped_metabolites,module_of_interest):
    
    '''
    This function summarises a module of interest with all the relevant data (features, mapped mets, adduts, rt, metabolite name)
    '''
    # m.z values:
    m_z = [float(x.split('_')[-1]) for x in expression_data.index.tolist()]
    
    # retention time:
    rt_dict = dict(zip(all_info_dat['mz_name'],all_info_dat['rt']))
    rt_vals = [rt_dict[x] for x in expression_data.index.tolist()]
    
    # Get p-values (0.049 when features are associated with module of interest)
    p_vals = np.ones(expression_data.shape[0])
    p_vals[df_modules[df_modules['Module']==int(module_of_interest)].index.tolist()] = 0.049
    
    # Get the membership values sorted:
    corr_vals = module_membership_data.loc[:,int(module_of_interest)]
    degree_vals = df_modules['Weighted.degree']

    outp_dataf = pd.DataFrame([expression_data.index.tolist(),m_z,rt_vals,corr_vals,degree_vals,p_vals]).T
    outp_dataf.columns = ['feature','m.z','r.t','corr.to.eigenvector','weighted.degree','p.value']
    
    # Now get matched "hits" from the metabolite dataframe:
    outp_dataf = pd.merge(outp_dataf, df_mapped_metabolites, on='feature', how='left')

    # Sort based on p-value:
    outp_dataf = outp_dataf.sort_values(by=['p.value','weighted.degree'],ascending=[True, False]).reset_index(drop = True)
    
    # Drop non-hitters:
    outp_dataf = outp_dataf.dropna(subset = ['metabolite_name'],axis = 0).reset_index(drop = True)
    
    # Drop the unknown_mz:
    outp_dataf = outp_dataf.drop(['unknown_mz'],axis = 1)
    return outp_dataf

def get_subset_pathways(annotated_module,pathbank_data,hmdb_or_kegg = 'HMDB'):
    
    '''
    This function creates a pathway dictionary where each pathway that is putatively annotated within a module
    maps to all its associated metabolite ids from pathbank data.
    '''
    
    dattt_subset = annotated_module[annotated_module['p.value'] < 0.05]
    if hmdb_or_kegg == 'HMDB':
        dat_subset = pathbank_data[pathbank_data['HMDB ID'].isin(dattt_subset['metabolite_id'].unique())].reset_index(drop = True)
        # Create a pathway dictionary:
        pathway_dict = {}

        # Use groupby to create a dictionary of pathway-metabolite associations
        pathway_dict = {group: data['HMDB ID'].tolist() for group, data in dat_subset.groupby('Pathway Name')}
    elif hmdb_or_kegg == 'KEGG':
        dat_subset = pathbank_data[pathbank_data['KEGG ID'].isin(dattt_subset['metabolite_id'].unique())].reset_index(drop = True)
        # Create a pathway dictionary:
        pathway_dict = {}

        # Use groupby to create a dictionary of pathway-metabolite associations
        pathway_dict = {group: data['KEGG ID'].tolist() for group, data in dat_subset.groupby('Pathway Name')}

    # Drop pathways that have less than 3 metabolite identifiers
    to_drop = []
    for key,val in pathway_dict.items():
        if len(val) < 1:
            to_drop.append(key)

    for pathway in to_drop:
        del pathway_dict[pathway]
    return pathway_dict


def pathway_enrichment_analysis(annotated_module,pathbank_data,hmdb_or_kegg = 'HMDB'):
    
    # START BY REMOVING SERINE FROM ANNOTATED MODULE:
    annotated_module = annotated_module[annotated_module['metabolite_id'] != 'HMDB0000187'].reset_index()
    
    # Assuming 'merged_df2' is your DataFrame with features and associated pathways
    if hmdb_or_kegg == 'HMDB':
        df_results2 = annotated_module.rename(columns = {'metabolite_id':'HMDB ID'})
        merged_df = pd.merge(df_results2, pathbank_data[['HMDB ID', 'Pathway Name']], on='HMDB ID', how='left')
        #print(merged_df)
    elif hmdb_or_kegg == 'KEGG':
        df_results2 = annotated_module.rename(columns = {'metabolite_id':'KEGG ID'})
        merged_df = pd.merge(df_results2, pathbank_data[['KEGG ID', 'Pathway Name']], on='KEGG ID', how='left')
    
    merged_df2 = merged_df.groupby('feature',sort = False)['Pathway Name'].apply(lambda x: [item for item in x if pd.notna(item)]).reset_index(name='pathways')
    df = merged_df2.copy()
    
    # List of pathways for analysis (Only do the ones that map to the hits of interest):
    pat_dict = get_subset_pathways(annotated_module,pathbank_data,hmdb_or_kegg = hmdb_or_kegg)
    #print(pat_dict)
    #print(len(pat_dict))
    if not pat_dict:
        print('Pathway dictionary is empty - no analysis is possible')
        results_df = pd.DataFrame()
        
    else:
        print('Annotating', len(pat_dict),'pathways...')
        pathways_to_test = list(pat_dict.keys())

        # DataFrame to store results
        results_list = []

        # Number of permutations for testing
        num_permutations = 1000

        # Specify the subset of features for analysis
        selected_features = list(np.unique(annotated_module[annotated_module['p.value'] < 0.05]['feature'].values))
        all_features = list(np.unique(annotated_module['feature'].values))

        for selected_pathway in pathways_to_test:

            # Observed count of selected pathway in the subset of features
            observed_count = np.sum(df[df['feature'].isin(selected_features)].apply(lambda x: selected_pathway in x['pathways'], axis=1))
            total_count_in_df = np.sum(df[df['feature'].isin(all_features)].apply(lambda x: selected_pathway in x['pathways'], axis=1))

            # Perform permutation testing
            permutation_counts = []
            for _ in range(num_permutations):
                # Shuffle the labels of the entire dataset
                shuffled_labels = np.random.permutation(df.index)
                #print(_)

                # Count the occurrences of the selected pathway in the shuffled labels
                shuffled_count = sum(df.loc[shuffled_labels[:len(selected_features)]].apply(lambda x: selected_pathway in x['pathways'], axis=1))
                permutation_counts.append(shuffled_count)

            # Calculate p-value based on permutation results
            p_value = (np.sum(np.array(permutation_counts) >= observed_count) + 1) / (num_permutations + 1)
            
            # Append results to the list
            results_list.append({
                'pathway': selected_pathway,
                'observed_count': observed_count,
                'observed_ratio':observed_count/total_count_in_df,
                'permutation_p.value': p_value,
            })

        # Convert the list to a DataFrame
        results_df = pd.DataFrame(results_list)
        results_df = results_df.sort_values(by = ['permutation_p.value'],ascending = [True]).reset_index(drop = True)
    return results_df
    
def create_accumulated_dict(series1,series2):
    out_dict = {}
    for indexx,element in enumerate(series1):
        if element not in out_dict:
            out_dict[element] = [series2[indexx]]
        else:
            out_dict[element].append(series2[indexx])
    return out_dict

def trim_module_dataframe_and_find_representative_features(eigendf,expression_data,correlation_df,df_modules,all_info,mz_to_mets):
    mods_to_remove = []
    module_dictionary_features = {}
    for modd in eigendf.columns.tolist():
        #print(modd)
        ann_mod = annotate_module_degree(expression_data,correlation_df,df_modules,all_info,mz_to_mets,modd)
        ann_mod = ann_mod[ann_mod['p.value'] < 1]
        values_to_find = ['M - H','M + H','M + Na','M - H2O + H','M + H2O - H','M + H2O + H','M - H2O - H']
        first_occurrence_index = ann_mod[ann_mod['adduct'].isin(values_to_find)].index.min()
        print('Module',modd)
        # Check if any of the values were found
        if pd.notna(first_occurrence_index):
            print(first_occurrence_index)
            chosen_feature = ann_mod.loc[first_occurrence_index,'feature']
            print(ann_mod.loc[first_occurrence_index,['feature','metabolite_name']].values)
            module_dictionary_features[modd] = chosen_feature
        else:
            mods_to_remove.append(modd)
            print(f"None of the specified values were found in the specified column.")
    return module_dictionary_features,mods_to_remove

## Annotation.

#### One can either use HMDB or KEGG for annotating unknown compounds. In this script, we will use HMDB. 

#### Load the PathBank database containing multiple metabolite identifiers mapping to pathways:

In [3]:
pathbank_dat = pd.read_csv('pathbank_all_metabolites.csv')

# Use only the human pathways:
pathbank_dat = pathbank_dat[pathbank_dat['Species'] == 'Homo sapiens'].reset_index(drop = True)

# Use only the metabolic pathway objects:
pathbank_dat = pathbank_dat[pathbank_dat['Pathway Subject'] == 'Metabolic'].reset_index(drop = True)

# Strip whitespace from the "KEGG ID" column
pathbank_dat['KEGG ID'] = pathbank_dat['KEGG ID'].str.strip()
pathbank_dat['HMDB ID'] = pathbank_dat['HMDB ID'].str.strip()

# Remove rows with NaN values
pathbank_dat = pathbank_dat.dropna(subset = ['KEGG ID'],axis = 0)

# Create a dictionary that maps KEGG ids to metabolite names (useful later):
kegg_dict = dict(zip(pathbank_dat['KEGG ID'],pathbank_dat['Metabolite Name']))
hmdb_dict = dict(zip(pathbank_dat['HMDB ID'],pathbank_dat['Metabolite Name']))

## Load the HMDB database to be able to scan and compare the unknown m/z values to known masses of HMDB compounds:

In [4]:
hmdb = pd.read_csv('hmdb.csv')
hmdb['name'] = hmdb['name'].apply(lambda x: x[2:-1])

# Generate the mass_to_kegg dict from the hmdb database:
mass_to_kegg_dict = {}

for index, row in hmdb.iterrows():
    mass = row['monisotopic_molecular_weight']
    kegg_id = row['kegg']

    if mass not in mass_to_kegg_dict:
        mass_to_kegg_dict[mass] = []

    mass_to_kegg_dict[mass].append(kegg_id)

mass_to_kegg_dict = {mass: [kegg_id for kegg_id in kegg_ids if pd.notna(kegg_id)]
                    for mass, kegg_ids in mass_to_kegg_dict.items()}
mass_to_kegg_dict = {mass: kegg_ids for mass, kegg_ids in mass_to_kegg_dict.items() if kegg_ids}

# Generate the mass_to_hmdb dict from the hmdb database:
mass_to_hmdb_dict = {}

for index, row in hmdb.iterrows():
    mass = row['monisotopic_molecular_weight']
    hmdb_id = row['accession']

    if mass not in mass_to_hmdb_dict:
        mass_to_hmdb_dict[mass] = []

    mass_to_hmdb_dict[mass].append(hmdb_id)

mass_to_hmdb_dict = {mass: [hmdb_id for hmdb_id in hmdb_ids if pd.notna(hmdb_id)]
                    for mass, hmdb_ids in mass_to_hmdb_dict.items()}
mass_to_hmdb_dict = {mass: hmdb_ids for mass, hmdb_ids in mass_to_hmdb_dict.items() if hmdb_ids}


C:\Users\sigur\AppData\Local\Temp\ipykernel_13120\2119306765.py:1: DtypeWarning: Columns (6,20) have mixed types. Specify dtype option on import or set low_memory=False.
  hmdb = pd.read_csv('hmdb.csv')


## Load relevant data:

In [228]:
correlation_df = pd.read_csv('correlation_df_JAN2024.csv',index_col = 0)
correlation_df.columns = correlation_df.columns.astype(int)

eigendf = pd.read_csv('eigendf_JAN2024.csv',index_col=0).reset_index(drop = True)
df_modules = pd.read_csv('df_modules_JAN2024.csv',index_col=0).reset_index(drop = True)
expression_data = pd.read_csv('All_data_normalised_21DEC2023.csv',index_col=0).T
all_info = pd.read_csv('xcms_output.csv',index_col=0).reset_index(drop = True)
all_adds = pd.read_csv('Adduct_df_JAN2024.csv',index_col = 0) # Adducts

## Prepare all feature masses from our LC-MS dataset:

In [216]:
unknown_masses = [float(x.split('_')[-1]) for x in expression_data.index.tolist()]
dict_type = {
    'neg':'negative',
    'pos':'positive',
    'bas':'negative'
}
vals = [dict_type[x.split('_')[0]] for x in expression_data.index.tolist()]
unknown_mass_dict = dict(zip(unknown_masses,zip(expression_data.index.tolist(),vals)))

## Match all our detected ions with compounds of same m/z in dataframe, with a ppm threshold of 5ppm:

In [ ]:
mz_to_mets = match_unknown_mz_values_with_metabolite_ids(unknown_mass_dict,mass_to_hmdb_dict,all_adds,hmdb_dict,ppm_threshold=5)

## Remove communities (modules) that do not have any putative annotation. Also, find the representative features for all modules (Protonated/Deprotonated parent ions, hydrated/dehydrated parent ions):

In [217]:
mod_dict,mods_remove = trim_module_dataframe_and_find_representative_features(eigendf,expression_data,correlation_df,df_modules,all_info,mz_to_mets)

Module 27
3
['bas_mz_119.0348069' '2-Ketobutyric acid']
Module 5
0
['neg_mz_208.0612357' '5-Hydroxyindoleacetic acid']
Module 8
1
['neg_mz_148.0425231' 'L-Methionine']
Module 15
1
['neg_mz_349.2728414' 'Adrenic acid']
Module 0
0
['neg_mz_163.039203' 'Phenylpyruvic acid']
Module 23
0
['neg_mz_185.0919051' 'Pyridoxamine']
Module 11
2
['neg_mz_367.1575865' 'Dehydroepiandrosterone sulfate']
Module 10
0
['pos_mz_166.0860973' 'L-Phenylalanine']
Module 20
1
['bas_mz_194.9456989' 'Pyrophosphate']
Module 9
0
['pos_mz_147.0438912' 'Phenylpyruvic acid']
Module 40
5
['pos_mz_120.0656837' 'L-Threonine']
Module 63
0
['bas_mz_263.1019867' 'Acetyl-N-formyl-5-methoxykynurenamine']
Module 6
0
['neg_mz_221.0589364' 'L-Cystathionine']
Module 7
0
['pos_mz_527.158164' 'Raffinose']
Module 68
1
['pos_mz_152.0705369' 'Norepinephrine']
Module 13
4
['bas_mz_283.2627966' 'Stearic acid']
Module 14
None of the specified values were found in the specified column.
Module 17
1
['neg_mz_367.1572477' 'Dehydroepiandroste

## Drop non-annotated communities (modules) from eigendf:

In [218]:
eigendf = eigendf.drop(mods_remove,axis = 1)
feature_df = expression_data.T[mod_dict.values()]

## Save for modeling:

In [230]:
eigendf.to_csv('eigendf_trimmed_JAN2024.csv')
feature_df.to_csv('feature_df_JAN2024.csv')
mz_to_mets.to_csv('mz_to_mets_JAN2024.csv')
pathbank_dat.to_csv('PathBank_data_JAN2024.csv')

with open('module_to_feature_dictionary.pkl', 'wb') as pickle_file:
    pickle.dump(mod_dict, pickle_file)